In [7]:
import numpy as np
import matplotlib.pyplot as plt
from hamiltonian_generator import Hamiltonian
import qutip as qt
from scipy.optimize import curve_fit

def calculate_dephasing_rate(A):
    # Set parameters based on the system from file_context_0
    phi_ex = 0.2
    Ej = 30.19 
    Ec = 0.1
    sc = Hamiltonian(phi_ex, Ej, Ec, [5,3])
    # Use the provided A value
    optimal_omega = sc.optimal_omegad(A)[0]*2*np.pi

    sc = Hamiltonian(phi_ex, Ej, Ec, [3,2])
    H0 = qt.Qobj(sc.H)
    H_control = qt.Qobj(sc.H_control)
    # Set up the Floquet system
    initial_state, state_and_sigmax, get_projector = sc.setup_floquet_system(A, optimal_omega)

    # Get the noise operator from the system
    t = 0
    U, _, _ = state_and_sigmax(t)
    Ud = U.T.conj()
    U,_,_ = state_and_sigmax(0)
    initial_state = qt.Qobj(U[:,1])

    # Parameters for time evolution
    gamma = 1/1000
    t_list = np.linspace(0, 3000, 5000)  # Time points for simulation (up to 2000 ns)

    # Use the system's Hamiltonian
    H = [H0, [H_control, lambda t, args: A * np.cos(optimal_omega * t)]]  # Time-dependent Hamiltonian

    # Create collapse operators using the system's noise operator
    L = np.sqrt(gamma) * qt.Qobj(sc.s)
    c_ops = [L]

    # Time evolution using mesolve
    result = qt.mesolve(
        H,
        initial_state,
        t_list,
        c_ops,
        options=qt.Options(rtol=1e-10, atol=1e-10)
    )

    dims = H0.full().shape[0]
    avg_values = np.zeros((dims+1, len(t_list)))
    for i, t in enumerate(t_list):
        # Get the state at time t
        state_t = result.states[i]
        
        # Get the projector at time t
        U, sigmaxc,_ = state_and_sigmax(t)
        projectors = get_projector(U)
        
        # Calculate expectation values for all projectors
        for j in range(dims):
            avg_values[j, i] = qt.expect(projectors[j], state_t)
        avg_values[dims,i] = qt.expect(qt.Qobj(sigmaxc), state_t)

    # Fit exponential decay to extract relaxation time
    def relaxation_fit(t,  T1):
        return  1 - np.exp(-t / T1)

    # Initial guess for parameters
    p0 = [1000]  # occupation and steady_time guess
    # Fit the data
    popt, pcov = curve_fit(relaxation_fit, t_list, avg_values[0] , p0=p0)
    T1 = popt[0]

    def steady_state_fit(t, occupation, steady_time):
        return occupation *(1 -  np.exp(-t / steady_time))
    # Initial guess for parameters
    p0 = [1e-3, 1000]  # occupation and steady_time guess
    # Fit the data
    popt, pcov = curve_fit(steady_state_fit, t_list, avg_values[3] , p0=p0)
    occupation_1 = popt[0]
    steady_time_1 = popt[1]

    # Get the noise operator from the system
    t = 0
    U, _, _ = state_and_sigmax(t)
    Ud = U.T.conj()
    U,_,_ = state_and_sigmax(0)
    initial_state = qt.Qobj(U[:,0])

    # Use the system's Hamiltonian
    H = [H0, [H_control, lambda t, args: A * np.cos(optimal_omega * t)]]  # Time-dependent Hamiltonian

    # Create collapse operators using the system's noise operator
    L = np.sqrt(gamma) * qt.Qobj(sc.s)
    c_ops = [L]

    # Time evolution using mesolve
    result = qt.mesolve(
        H,
        initial_state,
        t_list,
        c_ops,
        options=qt.Options(rtol=1e-10, atol=1e-10)
    )

    avg_values = np.zeros((dims+1, len(t_list)))
    for i, t in enumerate(t_list):
        # Get the state at time t
        state_t = result.states[i]
        
        # Get the projector at time t
        U, sigmaxc,_ = state_and_sigmax(t)
        projectors = get_projector(U)
        
        # Calculate expectation values for all projectors
        for j in range(dims):
            avg_values[j, i] = qt.expect(projectors[j], state_t)
        avg_values[dims,i] = qt.expect(qt.Qobj(sigmaxc), state_t)

    # Initial guess for parameters
    p0 = [1e-3, 1000]  # occupation and steady_time guess

    # Fit the data
    popt, pcov = curve_fit(steady_state_fit, t_list, avg_values[2], p0=p0)
    occupation_0 = popt[0]
    steady_time_0 = popt[1]

    initial_state, state_and_sigmax, get_projector = sc.setup_floquet_system(A, optimal_omega)

    initial_state = qt.Qobj(initial_state)

    # Parameters for time evolution
    gamma = 1/1000
    t_list = np.linspace(0, 10000, 50000)  # Time points for simulation (up to 2000 ns)

    # Use the system's Hamiltonian
    H = [H0, [H_control, lambda t, args: A * np.cos(optimal_omega * t)]]  # Time-dependent Hamiltonian

    # Create collapse operators using the system's noise operator
    L = np.sqrt(gamma) * qt.Qobj(sc.s)
    c_ops = [L]

    # Time evolution using mesolve
    result = qt.mesolve(
        H,
        initial_state,
        t_list,
        c_ops,
        options=qt.Options(rtol=1e-10, atol=1e-10)
    )

    dims = H0.full().shape[0]
    avg_values = np.zeros((dims+1, len(t_list)))
    for i, t in enumerate(t_list):
        # Get the state at time t
        state_t = result.states[i]
        
        # Get the projector at time t
        U, sigmaxc,_ = state_and_sigmax(t)
        projectors = get_projector(U)
        
        # Calculate expectation values for all projectors
        for j in range(dims):
            avg_values[j, i] = qt.expect(projectors[j], state_t)
        avg_values[dims,i] = qt.expect(qt.Qobj(sigmaxc), state_t)

    # Fit exponential decay to extract relaxation time
    def T2fit(t,  T2,omega):
        return np.exp(-t / T2)*np.cos(omega*t)

    # Initial guess for parameters
    p0 = [1e6, -sc.omega_c + optimal_omega]  # occupation and steady_time guess

    # Set bounds for parameters: T2 cannot be larger than 1e7
    bounds = ([0, -np.inf], [1e7, np.inf])  # (lower_bounds, upper_bounds)

    # Fit the data
    popt, pcov = curve_fit(T2fit, t_list, avg_values[-1], p0=p0, bounds=bounds)
    T2_fit = popt[0]
    omega_fit = popt[1]

    return (1/T2_fit - 1/T1/2)

# Example usage with A value
A = 1e-3 * 2 * np.pi  # Example A value
result = calculate_dephasing_rate(A)
result

np.float64(8.064873039391554e-07)

In [10]:
from hamiltonian_generator import Hamiltonian
import qutip as qt

def get_noise_operator(A):
    phi_ex = 0.2
    Ej = 30.19 
    Ec = 0.1
    
    sc = Hamiltonian(phi_ex, Ej, Ec, [5,4])
    optimal_omega = sc.optimal_omegad(A)[0]*2*np.pi
    sc = Hamiltonian(phi_ex, Ej, Ec, [3,2])
    initial_state, state_and_sigmax, get_projector = sc.setup_floquet_system(A, optimal_omega)
    
    t = 0
    U, _, _ = state_and_sigmax(t)
    Ud = U.T.conj()
    # Work with a fresh copy of noise every time
    noise1 = Ud @ sc.s @ U
    # Take the real part if necessary
    
    return noise1
def numerical_dephasing(A):
    noise = get_noise_operator(A)
    return 1e-3*np.abs(noise[1,1] - noise[0,0])**2/2 + 1e-3*np.abs(noise[3,1])**2/2 + 1e-3*np.abs(noise[2,0])**2/2

numerical_dephasing(A)



np.float64(7.994465466736493e-07)

np.float64(7.994465466736493e-07)